# Notebook for the systematic collection of DWD data

**Setup - once**

1. Install [MySQL](https://dev.mysql.com/doc/mysql-getting-started/en/): Comes with the MySQL workbench which allows you to manage data sets separately and test queries.
2. Install [PowerBI desktop](https://powerbi.microsoft.com/en-us/desktop/): for some visualization
3. Make sure you have the libraries installed for querying a DB:
```shell script
pip install mysql-connector-python
pip install SQLAlchemy
```

**Setup - every time you just started your jupyter notebook/lab**

Preparatory steps before being able to write to MySQL tables:
1. start your MySQL workbench
2. create / connect to your database in MySQL workbench
3. go to "Server" > "Startup/Shutdown" and click the "Start Server" button
4. enter your server credentials when prompted 
5. now you are ready to run queries over your data set from the workbench, this notebook or any script



**Objective of this notebook:**
1. extract/download all zip files and related text documents from a specific url like the annual historical data [here](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/annual/kl/historical/)
2. transform the relevant content from the zip files
3. load the content into a dedicated table in a dedicated MySQL dataset

**Data**

DWD climate data which is parsed in this notebook can be found [here](https://opendata.dwd.de/climate_environment/CDC/).

In [ ]:
import sys
sys.path.append("../")
from datapipeline import dwd
import logging

dwd.logger.setLevel(logging.DEBUG)
dwd.ch.setLevel(logging.DEBUG)

## Annual climate data

**Settings for measurements**

In [ ]:
dry = False
base_url = "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/annual/kl/historical/"
extraction_dir = "../data/climate_annual_kl_historical/extracted"
transformation_dir = "../data/climate_annual_kl_historical/transformed"
clean = False
db_creds_path = "../db_creds.json"
tmp_table = "tmp"
target_table = "measurements_annual"
dataset = "surreal_weather"
t_wait = .1
download_limit = 5

**Extraction**

In [ ]:
extraction = dwd.Extractor(base_url=base_url, extraction_dir=extraction_dir, dry=dry, t_wait=t_wait,
                       download_limit=download_limit)
extraction.run()

**Transform**

In [ ]:
transformation = dwd.Transformator(extraction_dir=extraction_dir, transformation_dir=transformation_dir, dry=dry,
                               target_table=target_table)
transformation.run(clean=clean)

**Loading**

[MySQL & pd.DataFrame](https://www.dataquest.io/blog/sql-insert-tutorial/)

[pd.DataFrame.to_sql docu](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html)

[sqlalchemy & connectors ](https://docs.sqlalchemy.org/en/13/dialects/mysql.html#module-sqlalchemy.dialects.mysql.mysqlconnector)

In [ ]:
loading = dwd.Loader(transformation_dir=transformation_dir, db_creds_path=db_creds_path, tmp_table=tmp_table,
                 target_table=target_table, dataset=dataset, dry=dry)
loading.run(clean=clean)

**Settings & loading of stations data (requires extraction and transform steps above)**

In [ ]:
dry = True
transformation_dir = "../data/climate_annual_kl_historical/transformed"
clean = False
db_creds_path = "../db_creds.json"
tmp_table = "tmp"
target_table = "stations_annual"
dataset = "surreal_weather"

In [ ]:
loading = dwd.Loader(transformation_dir=transformation_dir, db_creds_path=db_creds_path, tmp_table=tmp_table,
                 target_table=target_table, dataset=dataset, dry=dry)
loading.run(clean=clean)